In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from xgboost import XGBRegressor, XGBClassifier

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [14]:
# Loading data
train = pd.read_csv('../../data/train_data_1031.csv', index_col='id')
test = pd.read_csv('../../data/test_data_1031.csv', index_col='id')
X_train = train.drop(columns=['playtime_forever'])
X1_train = X_train
X2_train = train.loc[train['playtime_forever'] > 0, :].copy()
is_play = train['playtime_forever'].copy()
is_play.loc[is_play > 0] = 1
y1_train = is_play
y2_train = X2_train['playtime_forever']
X2_train = X2_train.drop(columns=['playtime_forever'])

In [15]:
xgbc = XGBClassifier()
xgbc.fit(X1_train, y1_train)
xgbr = XGBRegressor()
xgbr.fit(X2_train, y2_train)

[13:40:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [18]:
cross_val_score(xgbc, X1_train, y1_train, cv=10,  scoring='f1').mean()

0.7346390854304226

In [20]:
cross_val_score(xgbr, X2_train, y2_train, cv=10, scoring='neg_mean_squared_error').mean()

[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

-270.13188132625805

In [14]:
is_play = xgbc.predict(test)
playtime_forever = xgbr.predict(test)
submission = pd.DataFrame(data={
    'playtime_forever': playtime_forever,
    'is_play': is_play
})
submission.loc[submission['is_play'] == 0, 'playtime_forever'] = 0
submission.loc[submission['playtime_forever'] < 0, 'playtime_forever'] = 0
submission = submission.drop(columns=['is_play'])
submission.index.name = 'id'
submission.to_csv('../../submission/class_regression/xgb.csv')